In [6]:
# working with local tracking server

# importing the required libraries
import mlflow
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print("Tracking URI set to: ", mlflow.get_tracking_uri())

mlflow.search_experiments()

Tracking URI set to:  http://127.0.0.1:5000


[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748047947443, experiment_id='0', last_update_time=1748047947443, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
# creating a new experiment and run

mlflow.set_experiment("Iris_Classification_Scenario_2")

with mlflow.start_run():
    x, y = load_iris(return_X_y=True)

    parameters = {
        "C": 0.1,
        "max_iter": 100,
        "solver": "liblinear",
        "random_state": 42
    }

    linear_model = LogisticRegression(**parameters)
    linear_model.fit(x, y)
    
    # predicting the labels
    y_pred = linear_model.predict(x)
    accuracy = accuracy_score(y, y_pred)
    print(f"Accuracy: {accuracy}")

    # logging the model
    mlflow.log_metrics({"accuracy": accuracy})
    mlflow.log_params(parameters)

    mlflow.sklearn.log_model(linear_model, "models")
    print(f"Artifact URI: {mlflow.get_artifact_uri()}")

2025/05/24 03:01:12 INFO mlflow.tracking.fluent: Experiment with name 'Iris_Classification_Scenario_2' does not exist. Creating a new experiment.


Accuracy: 0.86


2025/05/24 03:01:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Artifact URI: mlflow-artifacts:/1/fb410d47314e4a209ac8c3f801a44434/artifacts
🏃 View run rare-shoat-35 at: http://127.0.0.1:5000/#/experiments/1/runs/fb410d47314e4a209ac8c3f801a44434
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [10]:
# model registry

from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")
client.search_registered_models()

[]

In [15]:
runs = client.search_runs(experiment_ids=["1"])
run_id = runs[0].info.run_id

mlflow.register_model(
    f"runs:/{run_id}/models",
    "Iris_Classification_Model"
)

Registered model 'Iris_Classification_Model' already exists. Creating a new version of this model...
2025/05/24 03:16:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Iris_Classification_Model, version 1
Created version '1' of model 'Iris_Classification_Model'.


<ModelVersion: aliases=[], creation_timestamp=1748049406650, current_stage='None', description='', last_updated_timestamp=1748049406650, name='Iris_Classification_Model', run_id='fb410d47314e4a209ac8c3f801a44434', run_link='', source='mlflow-artifacts:/1/fb410d47314e4a209ac8c3f801a44434/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>